In [30]:
# using sqlite3 for data exploration + visualization. pandas - for df
import sqlite3
import pandas as pd

In [31]:
# ipython-sql for sql magic statements, matplotlib and seaborn for visualization 
!pip install ipython-sql
!pip install seaborn 
import seaborn as sns
%load_ext sql
import matplotlib.pyplot as plt


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


The sql extension is already loaded. To reload it, use:
  %reload_ext sql



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [32]:
# this either creates a new database or uses the existing one if it exists. it's done in thee current directory
# here we make a new database - netflix.db
conn = sqlite3.connect('netflix.db')
cur = conn.cursor()

In [33]:
# check the current directory. That's where the database is stored
%pwd

'C:\\Users\\haykg\\Documents\\github\\Data-cleaning'

In [34]:
# this is to be able to use the netflix database with magic sql commands
%sql sqlite:///netflix.db

In [39]:
df = pd.read_csv('netflix_titles.csv')

In [43]:
# rows and columns of the dataframe
df.shape

(8807, 12)

In [41]:
# pandas method that takes the pandas DataFrame df and writes it into my SQL database and names Chicago_socioeconomic_data.
# index = FALSE says **Do not write the DataFrame’s index** as a column in the SQL table. By default, `to_sql` will create 
# a column for the DataFrame’s index; setting `index=False` skips that.
# method = "multi" will allow to insert several rows simultaneously. Otherwise it would just be one row at a time.
df.to_sql("netflix_raw", conn, if_exists="replace", index= "False")

8807

In [44]:
# For below you can also
# Install the 'ipython-sql' and 'prettytable' libraries using pip
!pip install ipython-sql prettytable

# Import the 'prettytable' library, which is used to display data in a formatted table
import prettytable

# Set the default display format for prettytable to 'DEFAULT' (i.e., a simple table format)
prettytable.DEFAULT = 'DEFAULT'


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [69]:
%sql select * from netflix_raw limit 2

 * sqlite:///netflix.db
Done.


show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,None,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmmaker Kirsten Johnson stages his death in inventive and comical ways to help them both face the inevitable."
s2,TV Show,Blood & Water,None,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thabang Molaba, Dillon Windvogel, Natasha Thahane, Arno Greeff, Xolile Tshabalala, Getmore Sithole, Cindy Mahlangu, Ryle De Morny, Greteli Fincham, Sello Maake Ka-Ncube, Odwa Gwanya, Mekaila Mathys, Sandi Schultz, Duane Williams, Shamilla Miller, Patrick Mofokeng",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town teen sets out to prove whether a private-school swimming star is her sister who was abducted at birth."


In [49]:
# checking the promary keys. We find there are no primary keys - so I will delete the useless "False" column and make show_id the main one.
%sql PRAGMA table_info(netflix_raw)

 * sqlite:///netflix.db
Done.


cid,name,type,notnull,dflt_value,pk
0,False,INTEGER,0,None,0
1,show_id,TEXT,0,None,0
2,type,TEXT,0,None,0
3,title,TEXT,0,None,0
4,director,TEXT,0,None,0
5,cast,TEXT,0,None,0
6,country,TEXT,0,None,0
7,date_added,TEXT,0,None,0
8,release_year,INTEGER,0,None,0
9,rating,TEXT,0,None,0


In [70]:
#drop the index that references the column "False" (check this out)
%sql DROP INDEX IF EXISTS "ix_netflix_raw_False";

# then drop the column
ALTER TABLE netflix_raw DROP COLUMN "False";

 * sqlite:///netflix.db
Done.


[]

In [71]:
%sql ALTER TABLE netflix_raw DROP COLUMN "False";

 * sqlite:///netflix.db
(sqlite3.OperationalError) no such column: "False"
[SQL: ALTER TABLE netflix_raw DROP COLUMN False ;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
